In [1]:
import os
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
import arabic_reshaper
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
arabic = lambda x: arabic_reshaper.reshape(x)[::-1]

In [2]:
model = tf.keras.models.load_model('saved_models/sign_letters_detection_03_points_01')
model.summary()
labels = [
	'ain', 'al', 'aleff','bb','dal','dha','dhad','fa','gaaf','ghain','ha','haa','jeem','kaaf','khaa','la','laam',
	'meem','nun','ra','saad','seen','sheen','ta','taa','thaa','thal','toot','waw','ya','yaa','zay'
	]
class_names = sorted(labels)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_85 (Dense)            (None, 128)               8192      
                                                                 
 dropout_51 (Dropout)        (None, 128)               0         
                                                                 
 dense_86 (Dense)            (None, 256)               33024     
                                                                 
 dropout_52 (Dropout)        (None, 256)               0         
                                                                 
 dense_87 (Dense)            (None, 256)               65792     
                                                                 
 dropout_53 (Dropout)        (None, 256)               0         
                                                                 
 dense_88 (Dense)            (None, 128)             

In [3]:
def get_modified_img(img, landmarks):
	mp_drawing.draw_landmarks(
		img,
		landmarks,
		mp_hands.HAND_CONNECTIONS,
		mp_drawing_styles.get_default_hand_landmarks_style(),
		mp_drawing_styles.get_default_hand_connections_style()
	)
	return cv2.flip(img, 1)

def get_points(hand):
	return [
		point.__getattribute__(attr)
			for attr in ['x', 'y', 'z']
				for point in hand.landmark
	]

def process_image(img):
	with mp_hands.Hands(
		static_image_mode=True,
		max_num_hands=2,
		min_detection_confidence=0.7) as hands:
			ann_img = cv2.flip(img.copy(), 1)
			results = hands.process(cv2.cvtColor(ann_img, cv2.COLOR_BGR2RGB))
	img_hands = results.multi_hand_landmarks
	if img_hands is None:
		return None
	modified_img = get_modified_img(cv2.flip(img.copy(), 1), img_hands[0])
	cv2.imshow('Modified-Image', modified_img)
	return get_points(img_hands[0])

In [12]:
def predict(cv2_img):
	# cv2.imshow("Camera-Saved", cv2_img)
	points = process_image(cv2_img)
	if not points:
		print("Could not detect a hand in this image !")
		return
	predictions = model.predict(np.array(points).reshape(1,len(points)))
	score = tf.nn.softmax(predictions[0])
	print(f"This image most likely belongs to {class_names[np.argmax(score)]} "
			f"with a {100 * np.max(score):.2f} percent confidence.")

In [13]:
cam = cv2.VideoCapture(0)
cv2.namedWindow("Camera-Live")
while True:
	_, frame = cam.read()
	cv2.imshow("Camera-Live", frame)
	key = cv2.waitKey(20)
	if key == 13: # enter
		# print("Enter detected ...")
		predict(frame)
	if key == 27: # escape
		print("Escape detected ...")
		break
cam.release()
cv2.destroyAllWindows()
print("Program ended.")

This image most likely belongs to toot with a 14.57 percent confidence.
This image most likely belongs to toot with a 11.53 percent confidence.
This image most likely belongs to toot with a 61.13 percent confidence.
This image most likely belongs to toot with a 69.01 percent confidence.
This image most likely belongs to zay with a 35.18 percent confidence.
This image most likely belongs to zay with a 8.94 percent confidence.
This image most likely belongs to toot with a 58.87 percent confidence.
This image most likely belongs to fa with a 91.92 percent confidence.
This image most likely belongs to ha with a 42.98 percent confidence.
This image most likely belongs to toot with a 36.68 percent confidence.
This image most likely belongs to gaaf with a 63.01 percent confidence.
This image most likely belongs to toot with a 11.94 percent confidence.
This image most likely belongs to ra with a 12.37 percent confidence.
This image most likely belongs to ra with a 9.03 percent confidence.
This